In [2]:
import pandas as pd 
from binance.client import Client
from binance_key import api_key , secret_key
from datetime import datetime , timedelta
import math
from binance.exceptions import *
import numpy as np

import time

In [3]:
client = Client(api_key , secret_key,tld ='us')


In [4]:
def truncate(number, decimals=0):
    """
    Returns a value truncated to a specific number of decimal places.
    https://stackoverflow.com/questions/783897/how-to-truncate-float-values
    credit: nullstellensatz
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

In [5]:
# calculating moving average
def sma(data , window):
    return (data.rolling(window = window).mean())

# bollinger band strat
def bollinger_bands(data , sma , window ,nstd):
#     standard deviation 
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band , lower_band

In [6]:
symbols = ['BTC', 'ETH','LTC']
start_n_hours_ago = 48

def gather_data(symbols, start_n_hours_ago):
    merge = False
    for symbol in symbols :
        
    #     start_date = "May 1, 2022"
        print(f'gathering {symbol}')
        klines = client.get_historical_klines(symbol = f'{symbol}USDT',
                                              interval = client.KLINE_INTERVAL_1HOUR, 
                                              start_str = str(datetime.now() - timedelta(hours = start_n_hours_ago)))

        # initializing the columns 
        cols = [
            'OpenTime',
            f'{symbol}-USD_Open',
            f'{symbol}-USD_High',
            f'{symbol}-USD_Low',
            f'{symbol}-USD_Close',
            f'{symbol}-USD_Volume',
            'CloseTime',
            f'{symbol}-QuoteAssetVolume',
            f'{symbol}-NumberOfTrades',
            f'{symbol}-TBBASV',
            f'{symbol}-TBQAV',
            f'{symbol}-ignore',

        ]

        #  setting the columns to cols 
        # klines.columns = cols 
        df = pd.DataFrame(klines ,columns  = cols)


        if merge == True:
            dfs = pd.merge(df ,dfs, how ='inner', on = ['OpenTime' , 'CloseTime'])
        else :
            dfs = df
            merge = True

    dfs.OpenTime = [datetime.fromtimestamp(ts / 1000) for ts in df.OpenTime]
    dfs.CloseTime = [datetime.fromtimestamp(ts /1000) for ts in dfs.CloseTime]

    for col in dfs.columns :
        if not 'Time' in col :
            dfs[col] = dfs[col].astype(float)

    for symbol in symbols :
        dfs[f'{symbol}_sma'] = sma(dfs[f'{symbol}-USD_Close'], 20)
        dfs[f'{symbol}_upper_band'] , dfs[f'{symbol}_lower_band'] = bollinger_bands(dfs[f'{symbol}-USD_Close'], 
                                                                               sma =  dfs[f'{symbol}_sma'],
                                                                               window =20, nstd=3)
    dfs.dropna(inplace =True)
        
    return dfs


In [7]:
# dfs.OpenTime = [datetime.fromtimestamp(ts / 1000) for ts in df.OpenTime]
# dfs.CloseTime = [datetime.fromtimestamp(ts /1000) for ts in dfs.CloseTime]

In [8]:
df =gather_data(symbols , 48)

gathering BTC
gathering ETH
gathering LTC


In [9]:
def get_states(df, symbols):
    states = {}

    for symbol in symbols :
        if df[f'{symbol}-USD_Close'].iloc[-1] <  df[f'{symbol}_lower_band'].iloc[-1] :
            states[symbol] = 'below'
        elif  df[f'{symbol}-USD_Open'].iloc[-1] > df[f'{symbol}_upper_band'].iloc[-1] :
            states[symbol] = 'above'
        else :
            states[symbol] = 'inside' 
            
    return states

In [11]:
#  for calculating precision 
precision = {}
for symbol in symbols :
    precision[symbol] = client.get_symbol_info(f'{symbol}USDT')['quotePrecision']
    

BUY_AMOUNT_USD =100  # starting price for buying the coins

balance_unit = 'USDT'
first = True
# for constantly gathering the live data

while True :
    if datetime.now().second% 10 == 0 and first :
        if (datetime.now().minute == 0 and datetime.now().second ==10) and first :
#             start gathering data at the top of hours
                first = False
                df  = gather_data(symbols , 48)
                states = get.states(df, symbols)
                print('current state of the market:')
                print(states)
                
        try:
            print('\n')
            if balance_unit == 'USDT':
                for symbol in symbols :
                    ask_price = float(client.get_orderbook_ticker(symbol = f'{symbol}USDT')['askPrice'])
                    lower_band =df[f'{symbol}_lower_band'].iloc[-1]

                    print( f'  {symbol}  :    ask price: {ask_price} and lower band :{lower_band}')

                    if ask_price < lower_band and states[symbol] == 'inside':      # buy signal
                        
                        buy_order = client.order_limit_buy(f'{symbol}-USDT',
                                                          quantity = truncate(BUY_AMOUNT_USD /ask_price, precision[symbol]),
                                                          price = ask_price)
                        print(buy_order)

                        start = datetime.now()
                        while True:
                            time.sleep(1)
                            buy_order = client.get_order(symbol=buy_order['symbol'], orderId=buy_order['orderId'])

                            seconds_since_buy = (datetime.now() - start).seconds

                            # resolve buy order
                            if float(buy_order['executedQty']) == 0 and seconds_since_buy > 60*60:
                                # no fill
                                client.cancel_order(symbol=buy_order['symbol'], orderId=buy_order['orderId'])
                                print('Order not filled after 1 hour, cancelled.')
                                print('\n')
                                break

                            if float(buy_order['executedQty']) != 0 and float(buy_order['executedQty']) != float(buy_order['origQty']) and seconds_since_buy > 60*60:
                                # partial fill
                                client.cancel_order(symbol=buy_order['symbol'], orderId=buy_order['orderId'])
                                balance_unit = symbol
                                print('Order partially filled after 1 hour, cancelled the rest and awaiting sell signal.')
                                print('\n')
                                break

                            if float(buy_order['executedQty']) ==  float(buy_order['origQty']):
                                # completely filled
                                balance_unit = symbol
                                print('Order filled:')
                                print(buy_order)
                                print('\n')
                                break

            if balance_unit != 'USDT':

                bid_price = float(client.get_orderbook_ticker(symbol = f'{balance_unit}USDT')['bidPrice'])
                upper_band =df[f'{balance_unit}_upper_band'].iloc[-1]

                print(f'{bid_price} and {upper_band}')

                if bid_price > upper_band and states[balance_unit] == 'inside':     #sell signal
                    print('sell')
                    balance_unit = 'USDT'

            time.sleep(1)
        
        except BinanceAPIException as e:
            print(e.status_code)
            print(e.message)
            
        



  BTC  :    ask price: 29781.95 and lower band :28720.26323629568
  ETH  :    ask price: 1974.61 and lower band :1916.0982057197875
  LTC  :    ask price: 69.56 and lower band :67.66801252580309


  BTC  :    ask price: 29781.95 and lower band :28720.26323629568
  ETH  :    ask price: 1974.61 and lower band :1916.0982057197875
  LTC  :    ask price: 69.56 and lower band :67.66801252580309


KeyboardInterrupt: 

In [1]:
states

NameError: name 'states' is not defined